# Pandas and duckdb

## Task 0

In [ ]:
import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

films.head()

In [ ]:
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df()

dfs.keys()

## Task 1

In [ ]:
long_movies = duckdb.sql("""
SELECT title, length AS duration
FROM films 
WHERE length > 180; 
""").df()

long_movies.head()

In [ ]:
love_word = duckdb.sql("""
SELECT title, rating, length, description 
FROM films
WHERE title ILIKE '%love%'; 
""").df()

love_word.head()

In [19]:
aggregate_function = duckdb.sql("""
SELECT 
    MIN(length) AS min_length_movies, 
    ROUND(AVG(length)) AS mean_length_movies, 
    MEDIAN(length) AS median_length_movies, 
    MAX(length) AS max_length_movies
FROM films
""").df()


aggregate_function.head()

,min_length_movies,mean_length_movies,median_length_movies,max_length_movies
0,46,115.0,114.0,185
